<a href="https://colab.research.google.com/github/EtienneFerrandi/Word-embedding/blob/main/Word_embedding_ngram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
torch.manual_seed(1)

In [ ]:
word_to_ix = {"hello": 0, "world": 1}
embeds = nn.Embedding(2, 5)
lookup_tensor = torch.tensor([word_to_ix["hello"]], dtype=torch.long)
hello_embed = embeds(lookup_tensor)
print(hello_embed)

tensor([[ 0.2857,  0.6898, -0.6331,  0.8795, -0.6842]],
       grad_fn=<EmbeddingBackward>)


In [ ]:
CONTEXT_SIZE = 2

In [ ]:
EMBEDDING_DIM = 10

In [ ]:
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

In [ ]:
ngrams = [
    (
        [test_sentence[i - j - 1] for j in range(CONTEXT_SIZE)],
        test_sentence[i]
    )
    for i in range(CONTEXT_SIZE, len(test_sentence))
]
print(ngrams[:3])

[(['forty', 'When'], 'winters'), (['winters', 'forty'], 'shall'), (['shall', 'winters'], 'besiege')]


In [ ]:
vocab = set(test_sentence)

In [ ]:
word_to_ix = {word: i for i, word in enumerate(vocab)}

In [ ]:
class NGramLanguageModeler(nn.Module):

  def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

  def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs


In [ ]:
losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [ ]:
for epoch in range(10):
    total_loss = 0
    for context, target in ngrams:
              context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)
              model.zero_grad()
              log_probs = model(context_idxs)
              loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))
              loss.backward()
              optimizer.step()
              total_loss += loss.item()
    losses.append(total_loss)
print(losses)

[516.1814589500427, 513.3693492412567, 510.57821679115295, 507.80692195892334, 505.05404806137085, 502.31887888908386, 499.59962725639343, 496.8926088809967, 494.198885679245, 491.5178225040436]


In [ ]:
print(model.embeddings.weight[word_to_ix["beauty"]])

tensor([-0.4497,  0.4590,  0.7382, -1.1819,  0.2708, -0.4747,  0.5774,  0.1755,
         1.0462, -1.5931], grad_fn=<SelectBackward>)
